## Conclusion - Final Solution

Let us review the Final Solution for our problem statement **daily_product_revenue**.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
val username = System.getProperty("user.name")

In [1]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate

username = itv002480
spark = org.apache.spark.sql.SparkSession@3f6ff3a8


org.apache.spark.sql.SparkSession@3f6ff3a8

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Prepare tables
  * Create tables
  * Load the data into tables
* We need to project the fields which we are interested in.
  * order_date
  * order_item_product_id
  * product_revenue
* As we have fields from multiple tables, we need to perform join after which we have to filter for COMPLETE or CLOSED orders.
* We have to group the data by order_date and order_item_product_id, then we have to perform aggregation on order_item_subtotal to get product_revenue.

In [4]:
%%sql
use itv002480_retail

Waiting for a Spark session to start...

++
||
++
++



In [5]:
%%sql

DROP DATABASE itv002480_retail CASCADE

++
||
++
++



In [6]:
%%sql

CREATE DATABASE IF NOT EXISTS itv002480_retail

++
||
++
++



In [7]:
%%sql

USE itv002480_retail

++
||
++
++



In [8]:
%%sql

SHOW tables

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [9]:
%%sql

CREATE TABLE orders (
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [10]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/orders' INTO TABLE orders

++
||
++
++



In [11]:
%%sql 

CREATE TABLE order_items (
    order_item_id INT,
    order_item_order_id INT,
    order_item_product_id INT,
    order_item_quantity INT,
    order_item_subtotal FLOAT,
    order_item_product_price FLOAT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

++
||
++
++



In [12]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items' INTO TABLE order_items

++
||
++
++



In [13]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS product_revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id

|2014-06-06 00:00:...|                  810|  ...


+--------------------+---------------------+---------------+
|          order_date|order_item_product_id|product_revenue|
+--------------------+---------------------+---------------+
|2014-03-28 00:00:...|                  793|          59.96|
|2014-04-09 00:00:...|                  191|        6599.34|
|2014-04-10 00:00:...|                  775|           9.99|
|2014-04-15 00:00:...|                  116|         404.91|
|2014-05-03 00:00:...|                  172|          120.0|
|2014-05-24 00:00:...|                  249|         164.91|
|2014-05-29 00:00:...|                 1014|        2698.92|
|2014-06-01 00:00:...|                 1073|        3199.84|
|2014-06-06 00:00:...|                  810|          39.98|
|2014-06-08 00:00:...|                  792|          89.94|
+--------------------+---------------------+---------------+
only showing top 10 rows



In [14]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS product_revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
ORDER BY o.order_date,
    product_revenue DESC

|2013-07-25 00:00:...|                  627|  ...


+--------------------+---------------------+---------------+
|          order_date|order_item_product_id|product_revenue|
+--------------------+---------------------+---------------+
|2013-07-25 00:00:...|                 1004|        5599.72|
|2013-07-25 00:00:...|                  191|        5099.49|
|2013-07-25 00:00:...|                  957|         4499.7|
|2013-07-25 00:00:...|                  365|        3359.44|
|2013-07-25 00:00:...|                 1073|        2999.85|
|2013-07-25 00:00:...|                 1014|        2798.88|
|2013-07-25 00:00:...|                  403|        1949.85|
|2013-07-25 00:00:...|                  502|         1650.0|
|2013-07-25 00:00:...|                  627|        1079.73|
|2013-07-25 00:00:...|                  226|         599.99|
+--------------------+---------------------+---------------+
only showing top 10 rows



* Using Spark SQL with Python or Scala

In [ ]:
spark.sql("DROP DATABASE itversity_retail CASCADE")

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS itversity_retail")

In [ ]:
spark.sql("USE itversity_retail")

In [ ]:
spark.sql("SHOW tables").show()

In [ ]:
spark.sql("""
CREATE TABLE orders (
    order_id INT,
    order_date STRING,
    order_customer_id INT,
    order_status STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")

In [ ]:
spark.sql("""
LOAD DATA LOCAL INPATH '/data/retail_db/orders' 
INTO TABLE orders
""")

In [ ]:
spark.sql("""
CREATE TABLE order_items (
    order_item_id INT,
    order_item_order_id INT,
    order_item_product_id INT,
    order_item_quantity INT,
    order_item_subtotal FLOAT,
    order_item_product_price FLOAT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")

In [ ]:
spark.sql("""
LOAD DATA LOCAL INPATH '/data/retail_db/order_items' 
INTO TABLE order_items
""")

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS product_revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
""").show()

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS product_revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
ORDER BY o.order_date,
    product_revenue DESC
""").show()